In [ ]:
from sqlalchemy import TextClause, Row, create_engine
import re

import pandas as pd
import sys


sys.path.insert(1, r"..\models")
sys.path.insert(2, r"..\etl")
sys.path.insert(3, r"..\predictions")
from src.models.lstm_model import load_model_weights_from_checkpoint
from connect_db import connect_to_grad_db
from preprocess import ready_text_for_pred
from tfidf_functions import *
from words_in_cloud import word_cloud


In [52]:

engine = connect_to_grad_db()


In [85]:

with engine.connect() as conn:
    query = TextClause("select * from news_api")
    content = conn.execute(query).fetchall()
  

In [88]:
df_news = pd.DataFrame(content)
df_news = df_news.dropna(subset=['source'])

In [86]:

df_news

,source,author,title,description,url,publishedAt,content,news_api_id,hottest_word_1,hottest_word_2,hottest_word_3,hottest_word_4,hottest_word_5


In [89]:
from tfidf_functions import *

df_hottest_word = hottest_word(df_news['content'])
df_hottest_word

c:\Users\fehmm\OneDrive\Skrivebord\python\graduation_project\src\predictions\..\models\tfidf_functions.py:24: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 24 of the file c:\Users\fehmm\OneDrive\Skrivebord\python\graduation_project\src\predictions\..\models\tfidf_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  review_text = BeautifulSoup(raw_review).get_text()


,hottest_word_1,hottest_word_2,hottest_word_3,hottest_word_4,hottest_word_5
0,party,election,said,leader,next
1,college,hospital,northern,july,student
2,party,election,said,leader,next
3,eu,russia,ukraine,sanctions,additional
4,jan,michigan,police,charge,thought
...,...,...,...,...,...
595,article,reuters,previous,adding,close
596,balloons,balloon,shot,military,article
597,welch,getty,images,film,ad
598,post,shared,love,heart,red


In [90]:

df_news.to_sql('news_api', con=engine, if_exists='append', index=False)

600

In [72]:
df_concat

,source,author,title,description,url,publishedAt,content,news_api_id,hottest_word_1,hottest_word_2,hottest_word_3,hottest_word_4,hottest_word_5
0,abc-news,JILL LAWLESS Associated Press,UK’s Labour Party says ex-leader Corbyn can’t ...,The leader of Britain's Labour Party says form...,https://abcnews.go.com/International/wireStory...,2023-02-15 12:33:54+00:00,LONDON -- Former U.K. opposition leader Jeremy...,1.0,party,election,said,leader,next
1,cnn,"Kocha Olarn, Chris Liakos, Sophie Tanno",One of 12 boys rescued from a Thai cave in 201...,"Duangphet Phromthep, one of the 12 boys rescue...",https://www.cnn.com/2023/02/15/europe/duangphe...,2023-02-15 12:29:10+00:00,\n\n\n\n\n\n\n\n\n\nDuangpetch Phromthep speak...,2.0,college,hospital,northern,july,student
2,abc-news,JILL LAWLESS Associated Press,UK’s Labour Party says ex-leader Corbyn can’t ...,The leader of Britain's Labour Party says form...,https://abcnews.go.com/Politics/wireStory/uks-...,2023-02-15 12:25:07+00:00,LONDON -- Former U.K. opposition leader Jeremy...,3.0,party,election,said,leader,next
3,abc-news,RAF CASERT Associated Press,"EU seeks new Russia sanctions package, targets...",The European Union is considering a new set of...,https://abcnews.go.com/Politics/wireStory/eu-s...,2023-02-15 12:25:05+00:00,BRUSSELS -- The European Union is considering ...,4.0,eu,russia,ukraine,sanctions,additional
4,abc-news,CHALIDA EKVITTAYAVECHNUKUL Associated Press,Thai suspect in Michigan hit-and-run agrees to...,A Thai-American woman living in Michigan who f...,https://abcnews.go.com/International/wireStory...,2023-02-15 12:21:08+00:00,BANGKOK -- A Thai-American woman living in Mic...,5.0,jan,michigan,police,charge,thought
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,None,None,None,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN
1796,None,None,None,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN
1797,None,None,None,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN
1798,None,None,None,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

df_clean = [ready_text_for_pred(text) for text in df['comment_text']]

# print(ready_text_for_pred(df['comment_text'][0]))

model = load_model_weights_from_checkpoint()



In [ ]:
pred_text = model.predict(df_clean)

pred_scam = model.predict(["tesla is an obvious scam"])


In [ ]:
pred_scam

In [ ]:
stats = {sentence:pred for sentence, pred in zip(df['comment_text'], pred_text)}


In [ ]:
df_results = pd.DataFrame(
    {
        "sentence":df['comment_text'],
        "prediction":pred_text.flatten()
    }
)

In [ ]:
df_results